In [2]:
import os, json, time, datetime as dt, csv, pathlib
from typing import Dict, List
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

DATA_RAW = pathlib.Path("d:/文心远/研究生/5040-Bootcamp/project/data/raw")
DATA_RAW.mkdir(parents=True, exist_ok=True)

load_dotenv()
ALPHA_KEY = os.getenv("ALPHAVANTAGE_API_KEY")
print("Loaded ALPHAVANTAGE_API_KEY?", bool(ALPHA_KEY))

d:\Software\Miniconda3\envs\fe-course\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Loaded ALPHAVANTAGE_API_KEY? True


Functions

In [3]:
def safe_stamp():
    return dt.datetime.now().strftime("%Y%m%d-%H%M%S")

def safe_filename(prefix: str, meta: Dict[str, str]) -> str:
    mid = "_".join([f"{k}-{str(v).replace(' ', '-')[:20]}" for k, v in meta.items()])
    return f"{prefix}_{mid}_{safe_stamp()}.csv"

def validate_df(df: pd.DataFrame, required_cols: List[str], dtypes_map: Dict[str, str]) -> Dict[str, str]:
    msgs = {}
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        msgs['missing_cols'] = f"Missing columns: {missing}"
    for col, dtype in dtypes_map.items():
        if col in df.columns:
            try:
                if dtype == 'datetime64[ns]':
                    pd.to_datetime(df[col])
                elif dtype == 'float':
                    pd.to_numeric(df[col])
            except Exception as e:
                msgs[f'dtype_{col}'] = f"Failed to coerce {col} to {dtype}: {e}"
    na_counts = df.isna().sum().sum()
    msgs['na_total'] = f"Total NA values: {na_counts}"
    return msgs

API Ingestion

In [4]:
SYMBOL = "MSFT"
use_alpha = bool(ALPHA_KEY)
print("Using Alpha Vantage:", use_alpha)

if use_alpha:
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY_ADJUSTED",
        "symbol": SYMBOL,
        "outputsize": "compact",
        "apikey": ALPHA_KEY,
        "datatype": "json"
    }
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    key = [k for k in js.keys() if "Time Series" in k][0]
    series = js[key]
    df_api = pd.DataFrame(series).T.reset_index().rename(columns={'index':'date', '5. adjusted close':'adj_close'})
    df_api['date'] = pd.to_datetime(df_api['date'])
    df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
else:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period="6mo", interval="1d", auto_adjust=True).reset_index()
    df_api = df_api[['Date','Close']]
    df_api.columns = ['date','adj_close']

df_api = df_api.sort_values('date').reset_index(drop=True)


Using Alpha Vantage: True


IndexError: list index out of range

In [ ]:
msgs = validate_df(df_api, required_cols=['date','adj_close'], dtypes_map={'date':'datetime64[ns]','adj_close':'float'})
print(msgs)

fname = safe_filename(prefix="api", meta={"source": "alpha" if use_alpha else "yfinance", "symbol": SYMBOL})
out_path = DATA_RAW / fname
df_api.to_csv(out_path, index=False)
print("Saved:", out_path)

{'na_total': 'Total NA values: 0'}
Saved: d:\文心远\研究生\5040-Bootcamp\project\data\raw\api_source-yfinance_symbol-MSFT_20250820-200354.csv
